In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd
from xgboost import XGBRegressor

In [2]:
data = pd.read_csv(r"binaaz_train.csv.zip")

In [3]:
data['area'] = data['Sahə'].str.split().str[0].astype(float)
data['rel_floor'] = data['Mərtəbə'].apply(lambda x: int(x.split('/')[0])/int(x.split('/')[1]))
data['Bina']= data['Mərtəbə'].str.split().str[2].astype('float')
data['Mərtəbə']= data['Mərtəbə'].str.split().str[0].astype('float')
data['doc'] = pd.get_dummies(data['Kupça'], drop_first=True)
data['mort'] = pd.get_dummies(data['İpoteka'], drop_first=True)
data['seher'] = data['seher'].astype('category').cat.codes
data['urg'] = data['description'].apply(lambda x: any([w in str(x).lower() for w in ['tecili', 'təcili', 'tacili', 'tcili', 'təcılı','срочно']]))

In [4]:
data['cond'] = data['description'].apply(lambda x: any([w in str(x).lower() for w in ['temirli', 'təmirli', 'tmrli','remontlu','remont','ремонтом','temir','təmir','əşya','esya','esyali','əşyalı','elverisli','əlverişli','muhafize','muhavize','mühafizə','qaraj','отремонтированный','yuksek','yüksək','super','ela','əla','qaz','qazli','qazlı']]))
data['new'] = data['title'].apply(lambda x: any([w in str(x).lower() for w in ['yeni', 'yni', 'новый']]))
data['cent']= data['description'].apply(lambda x: any([w in str(x).lower() for w in ['iceri seher','icheri sheher','iceriseher','icherisheher','içəri şəhər','içərişəhər', '28may','28 may','28 mayda','tarqovu','tarqovi','nizami','центр','sahil']]))
data['sea'] = data['description'].apply(lambda x: any([w in str(x).lower() for w in ['deniz', 'dəniz', 'dniz','море']]))
data['qesebe']= data['description'].apply(lambda x: any([w in str(x).lower() for w in ['qesebesi', 'qəsəbəsi', 'qəsəbə','поселке']]))
data['lux']=data['description'].apply(lambda x: any([w in str(x).lower() for w in ['premium', 'Premium', 'vip', 'Vip', 'VIP','super','Super','yuksek','Yüksək','yüksək','luks','Luks','lüks','Lüks',"əla","gözəl",'yüksək','keyfiyyətli',"bahalı"]]))
data['metro'] = data['description'].apply(lambda x: any([w in str(x).lower() for w in ['метро',"metro","metrosu",'metroya','metrosuna']]))


In [9]:
data = data[(data['area']>10)&(data['area']<1000)]
data = data[data['Otaq sayı']<15]
data = data[(data['price']>5000)&(data['price']<3000000)]
data = data[(data['price']/data['area']>500)&(data['price']/data['area']<3000)]
data = data[(data['area']/data['Otaq sayı']>5)&(data['area']/data['Otaq sayı']<100)]

In [6]:
data = data[(data['area']>10)&(data['area']<1000)]
data = data[data['Otaq sayı']<15]
data = data[(data['price']>5000)&(data['price']<3000000)]
data = data[(data['price']/data['area']>200)&(data['price']/data['area']<4000)]
data = data[(data['area']/data['Otaq sayı']>5)&(data['area']/data['Otaq sayı']<100)]

In [63]:
X = data[['Otaq sayı', 'area', 'rel_floor', 'doc', 'mort', 'urg','Mərtəbə',
         'Bina','latitude','longitude','cent','cond','new','seher']]
y = data['price']

In [8]:
data_test = pd.read_csv(r"binaaz_test.csv.zip")
data_test.head()

,_id,title,price_currency,poster,poster_type,Ünvan,description,Yeniləndi,Mərtəbə,Sahə,Otaq sayı,Kupça,İpoteka,locations,latitude,longitude,seher
0,20886,"Satılır 1 otaqlı yeni tikili 50.4 m², Koroğlu m.",AZN,Feqan,vasitəçi (agent),"Bakı şəhəri, Qara Qarayev pr.",TECILI...SATILIR...Nizami rayonu Qara qarayev ...,20 Yanvar 2021,8 / 14,50.4 m²,1,yoxdur,yoxdur,\nKoroğlu m.\nNizami r.\n\n,40.422748,49.923277,baki
1,117465,"Satılır 3 otaqlı yeni tikili 98 m², Badamdar q.",AZN,Baloğlan,mülkiyyətçi,"Bakı şəhəri, Göyüş Qarayev küçəsi",Badamdar qəsəbəsinin mərkəzində Araz supermark...,31 May 2021,5 / 16,98 m²,3,var,var,\nSəbail r.\nBadamdar q.\n\n,40.338863,49.806822,baki
2,125220,"Satılır 5 otaqlı yeni tikili 205 m², Yasamal r.",AZN,Əli,vasitəçi (agent),"Bakı şəhəri, Zahid Xəlilov küçəsi","Mənzil Yasamal rayonu, Zahid Xəlilov küçəsində...",10 Iyun 2021,13 / 18,205 m²,5,yoxdur,yoxdur,\nYasamal r.\n\n,40.382519,49.811850,baki
3,77683,"Satılır 3 otaqlı yeni tikili 117 m², Yeni Yasa...",AZN,Saya,vasitəçi (agent),"Bakı şəhəri, Əsəd Əhmədov küçəsi","Yasamal rayonu, Yeni Yasamal qesebesi,Esed Ehm...",16 Aprel 2021,4 / 16,117 m²,3,var,yoxdur,\nYasamal r.\nYeni Yasamal q.\n\n,40.390666,49.788563,baki
4,77672,"Satılır 4 otaqlı yeni tikili 227 m², Nəsimi r.",AZN,İlqar,vasitəçi (agent),"Bakı şəhəri, Hənifə Ələsgərova küçəsi.","Nəsimi rayonu, Hənifə Ələsgərova küçəsi Adora ...",16 Aprel 2021,9 / 18,227 m²,4,yoxdur,yoxdur,\nKoala parkı \nNəsimi bazarı \nZoopark \nDost...,40.393495,49.843594,baki


In [9]:
data_test['area'] = data_test['Sahə'].str.split().str[0].astype(float)
data_test['rel_floor'] = data_test['Mərtəbə'].apply(lambda x: int(x.split('/')[0])/int(x.split('/')[1]))
data_test['Bina']= data_test['Mərtəbə'].str.split().str[2].astype('float')
data_test['Mərtəbə']= data_test['Mərtəbə'].str.split().str[0].astype('float')
data_test['doc'] = pd.get_dummies(data_test['Kupça'], drop_first=True)
data_test['mort'] = pd.get_dummies(data_test['İpoteka'], drop_first=True)
data_test['urg'] = data_test['description'].apply(lambda x: any([w in str(x).lower() for w in ['tecili', 'təcili', 'tacili', 'tcili', 'təcılı','срочно']]))

In [10]:
data_test['cond'] = data_test['description'].apply(lambda x: any([w in str(x).lower() for w in ['temirli', 'təmirli', 'tmrli','remontlu','remont','ремонтом','temir','təmir','əşya','esya','esyali','əşyalı','elverisli','əlverişli','muhafize','muhavize','mühafizə','qaraj','отремонтированный','yuksek','yüksək','super','ela','əla','qaz','qazli','qazlı']]))
data_test['new'] = data_test['title'].apply(lambda x: any([w in str(x).lower() for w in ['yeni', 'yni', 'новый']]))
data_test['cent']= data_test['description'].apply(lambda x: any([w in str(x).lower() for w in ['iceri seher','icheri sheher','iceriseher','icherisheher','içəri şəhər','içərişəhər', '28may','28 may','28 mayda','tarqovu','tarqovi','nizami','центр','sahil']]))
data_test['sea'] = data_test['description'].apply(lambda x: any([w in str(x).lower() for w in ['deniz', 'dəniz', 'dniz','море']]))
data_test['qesebe']= data_test['description'].apply(lambda x: any([w in str(x).lower() for w in ['qesebesi', 'qəsəbəsi', 'qəsəbə','поселке']]))
data_test['lux']=data_test['description'].apply(lambda x: any([w in str(x).lower() for w in ['premium', 'Premium', 'vip', 'Vip', 'VIP','super','Super','yuksek','Yüksək','yüksək','luks','Luks','lüks','Lüks',"əla","gözəl",'yüksək','keyfiyyətli',"bahalı"]]))
data_test['metro'] = data_test['description'].apply(lambda x: any([w in str(x).lower() for w in ['метро',"metro","metrosu",'metroya','metrosuna']]))


In [11]:
data_test['seher'] = data_test['seher'].astype('category').cat.codes

In [12]:
data_test.head()

,_id,title,price_currency,poster,poster_type,Ünvan,description,Yeniləndi,Mərtəbə,Sahə,...,doc,mort,urg,cond,new,cent,sea,qesebe,lux,metro
0,20886,"Satılır 1 otaqlı yeni tikili 50.4 m², Koroğlu m.",AZN,Feqan,vasitəçi (agent),"Bakı şəhəri, Qara Qarayev pr.",TECILI...SATILIR...Nizami rayonu Qara qarayev ...,20 Yanvar 2021,8.0,50.4 m²,...,1,1,True,False,True,True,False,False,False,True
1,117465,"Satılır 3 otaqlı yeni tikili 98 m², Badamdar q.",AZN,Baloğlan,mülkiyyətçi,"Bakı şəhəri, Göyüş Qarayev küçəsi",Badamdar qəsəbəsinin mərkəzində Araz supermark...,31 May 2021,5.0,98 m²,...,0,0,False,True,True,True,False,True,True,False
2,125220,"Satılır 5 otaqlı yeni tikili 205 m², Yasamal r.",AZN,Əli,vasitəçi (agent),"Bakı şəhəri, Zahid Xəlilov küçəsi","Mənzil Yasamal rayonu, Zahid Xəlilov küçəsində...",10 Iyun 2021,13.0,205 m²,...,1,1,False,True,True,False,True,False,True,False
3,77683,"Satılır 3 otaqlı yeni tikili 117 m², Yeni Yasa...",AZN,Saya,vasitəçi (agent),"Bakı şəhəri, Əsəd Əhmədov küçəsi","Yasamal rayonu, Yeni Yasamal qesebesi,Esed Ehm...",16 Aprel 2021,4.0,117 m²,...,0,1,False,True,True,False,False,True,False,False
4,77672,"Satılır 4 otaqlı yeni tikili 227 m², Nəsimi r.",AZN,İlqar,vasitəçi (agent),"Bakı şəhəri, Hənifə Ələsgərova küçəsi.","Nəsimi rayonu, Hənifə Ələsgərova küçəsi Adora ...",16 Aprel 2021,9.0,227 m²,...,1,1,False,True,True,False,False,False,True,False


In [60]:
X_test = data_test[['Otaq sayı', 'area', 'rel_floor', 'doc', 'mort', 'urg','Mərtəbə',
         'Bina','latitude','longitude','cent','cond','new','seher']]

# XGBOOST

In [64]:
model_xgb = XGBRegressor(colsample_bytree = 1,learning_rate = 0.1,
max_depth=10, n_estimators=500, objective = 'reg:squarederror', subsample = 0.7)
model_xgb.fit(X, y,
              eval_set=[(X, y)],
              eval_metric=["rmse"],
              verbose=True)

[0]	validation_0-rmse:200559.68773
[1]	validation_0-rmse:181883.74129
[2]	validation_0-rmse:165153.06800
[3]	validation_0-rmse:150141.32508
[4]	validation_0-rmse:136666.06136
[5]	validation_0-rmse:124647.68094
[6]	validation_0-rmse:113837.30825
[7]	validation_0-rmse:104195.37897
[8]	validation_0-rmse:95562.66631
[9]	validation_0-rmse:87934.75272
[10]	validation_0-rmse:81126.77265
[11]	validation_0-rmse:75013.44065
[12]	validation_0-rmse:69645.38829
[13]	validation_0-rmse:64866.40797
[14]	validation_0-rmse:60609.92378
[15]	validation_0-rmse:56899.53326
[16]	validation_0-rmse:53590.11656
[17]	validation_0-rmse:50754.52243
[18]	validation_0-rmse:48150.80980
[19]	validation_0-rmse:45862.82564
[20]	validation_0-rmse:43867.85198
[21]	validation_0-rmse:42154.85226
[22]	validation_0-rmse:40698.92256
[23]	validation_0-rmse:39374.86324
[24]	validation_0-rmse:38259.97422
[25]	validation_0-rmse:37140.73786
[26]	validation_0-rmse:36176.26679
[27]	validation_0-rmse:35331.63684
[28]	validation_0-rmse

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=10, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [65]:
data_test['price'] = model_xgb.predict(X_test)
model1 = model_xgb.predict(X_test)

In [66]:
data_test[['_id', 'price']].to_csv('binaaz.csv', index=False)

# Random Forest

In [67]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=150,min_samples_split=2,min_samples_leaf= 1,max_features='sqrt',max_depth=20,bootstrap= False)
rf.fit(X,y)

RandomForestRegressor(bootstrap=False, max_depth=20, max_features='sqrt',
                      n_estimators=150)

In [68]:
x_train_pred=rf.predict(X)
rf.score(X,y)

0.9847259060883266

In [71]:
data_test['price']=rf.predict(X_test)
model2 = rf.predict(X_test)

In [72]:
data_test[['_id', 'price']].to_csv('binaaz.csv', index=False)

## Stacked 

In [70]:
data_test['price']=(model1+model2)/2

data_test[['_id', 'price']].to_csv('binaaz.csv', index=False)